# Interface Demonstration Notebook

A notebook to demonstrate an interface with some of the features of the Probability of Failure Model

Author: gavin.treseder@essentialenergy.com.au

In [1]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
from jupyter_dash import JupyterDash
import pandas as pd

import copy

from pof.component import Component
from pof.failure_mode import FailureMode
from interface.layouts import *

In [35]:
comp = Component().set_demo()
fm = FailureMode().set_demo()

In [36]:
# Component layout
mcl = make_component_layout(comp)
mfml = make_failure_mode_layout(fm)

In [37]:
graph_limit =  dbc.InputGroup(
    [
        dbc.InputGroupAddon(
            [
                dbc.Checkbox(id='graph_y_limit_active', checked=True),
                
            ],
            addon_type="prepend"
        ),
        dbc.Label("Graph Y Limit"),
        dbc.Input(
            type="number",
            id= 'graph_y_limit',
            #value = ,
            debounce=True,
        ),
    ],
)

graph_inputs = [Input('graph_y_limit_active', 'checked'), Input('graph_y_limit', 'value')]

In [48]:

# Build App
external_stylesheets = [dbc.themes.BOOTSTRAP]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Layout
app.layout = html.Div([
    html.Div(children='Test Output', id='test_output'),
    graph_limit,
    dcc.Graph(id="maintenance_strategy"),
    mcl,
])

collapse_ids = comp.get_objects()

@app.callback(
    [Output(f"{prefix}-collapse", "is_open") for prefix in collapse_ids],
    [Input(f"{prefix}-collapse-button", "n_clicks") for prefix in collapse_ids],
    [State(f"{prefix}-collapse", "is_open") for prefix in collapse_ids],
)
def toggle_collapses(*args):
    ctx = dash.callback_context

    state_id = ""
    collapse_id = ctx.triggered[0]['prop_id'].split('.')[0].replace('-collapse-button','')
    if collapse_id in collapse_ids: #TODO change to is not None

        state_id = collapse_id + '-collapse.is_open'
        ctx.states[state_id] = not ctx.states[state_id]

    is_open = tuple(ctx.states.values())

    return is_open

ms_fig_update = comp.get_dash_ids(prefix ='')

@app.callback(
    Output("maintenance_strategy", "figure"), 
    graph_inputs + [Input(dash_id,"checked") if 'active' in dash_id else Input(dash_id,"value") for dash_id in ms_fig_update]
)
def update_maintenance_strategy(graph_y_limit_active, graph_y_limit, *args):

    # Check the parameters that changed
    ctx = dash.callback_context
    dash_id = ctx.triggered[0]['prop_id'].split('.')[0]
    value = ctx.triggered[0]['value']

    """#Temp utnil mutliple failure modes are set up
    dash_id = dash_id.replace('comp-', "")

    # Update the model
    comp.dash_update(dash_id, value)

    comp_local = copy.deepcopy(comp)

    comp_local.mc_timeline(t_end=200, n_iterations=100)
    df = comp_local.expected_risk_cost_df(t_end=200)"""

    # update the model
    #dash_id = dash_id.replace('fm-', '')
    comp.dash_update(dash_id, value)
    # Simulate an outcome
    local = copy.deepcopy(comp)
    local.mc_timeline(t_end=200, n_iterations=100)
    df = local.expected_risk_cost_df()

    fig = px.area(
        df,
        x="time",
        y="cost_cumulative",
        color="task",
        line_group='failure_mode',
        title="Maintenance Strategy Costs",
    )

    if graph_y_limit_active:
        fig.update_yaxes(range=[0, graph_y_limit])

    return fig

"""@app.callback(
    Output("test_output", "children"),
    [Input('comp-active', 'checked')]
)
def test(task_value):
    # Check the parameters that changed
    ctx = dash.callback_context
    dash_id = ctx.triggered[0]['prop_id'].split('.')[0]
    value = ctx.triggered[0]['value']

    return "value" + str(value)"""

#Execute
app.run_server(mode='inline', debug=True)

In [47]:
comp.mc_timeline(100)
comp.expected_risk_cost_df()

100%|██████████| 100/100 [00:01<00:00, 53.98it/s]


,failure_mode,task,time,cost,cost_cumulative
0,random,inspection,0,0,0
0,random,inspection,1,0,0
0,random,inspection,2,0,0
0,random,inspection,3,0,0
0,random,inspection,4,0,0
...,...,...,...,...,...
11,fast_aging,risk,96,0,6500
11,fast_aging,risk,97,500,7000
11,fast_aging,risk,98,500,7500
11,fast_aging,risk,99,0,7500


In [13]:
from helper import fill_blanks

rc = fm.expected_risk_cost()

df = pd.DataFrame(rc).T.apply(fill_blanks, axis = 1, args = (0, 200))
df.index.name = 'task'
df_cost = df.explode('cost')['cost']
df = df.explode('time')
df['cost'] = df_cost

# Add a cumulative cost
df['cost_cumulative'] = df.groupby(by=['task'])['cost'].transform(pd.Series.cumsum)
df

,time,cost,cost_cumulative
task,,,
inspection,0,0,0
inspection,1,0,0
inspection,2,0,0
inspection,3,0,0
inspection,4,0,0
...,...,...,...
risk,196,0,5500
risk,197,0,5500
risk,198,0,5500


In [10]:
df

,time,cost
inspection,0,0
inspection,1,0
inspection,2,0
inspection,3,0
inspection,4,0
...,...,...
risk,196,0
risk,197,0
risk,198,0
risk,199,0


In [16]:
# Testing to find the problem with the 
comp.expected_risk_cost_df(t_end=200).groupby(by=['failure_mode', 'task']).sum()

time   cost  cost_cumulative
failure_mode task                                              
fast_aging   cm                   20100   4750           300250
             inspection           20100   2000           197000
             on_condition_repair  20100    249            24654
             risk                 20100  45000          3020000
random       cm                   20100   2350           283400
             inspection           20100   2000           197000
             on_condition_repair  20100      0                0
             risk                 20100  19000          2465000
slow_aging   cm                   20100   1650            93150
             inspection           20100   2000           197000
             on_condition_repair  20100    145            14503
             risk                 20100  16500           963000

In [23]:
for fm in comp.fm.values():
    fm.cof.risk_cost_total = 10000

In [23]:
def update_maintenance_strategy():

    comp.mc_timeline(t_end=200, n_iterations=100)
    df = comp.expected_risk_cost_df(t_end=200)

    fig = px.area(
        df,
        x="time",
        y="cost_cumulative",
        color="task",
        line_group='failure_mode',
        title="Maintenance Strategy Costs",
    )

    return fig

update_maintenance_strategy().show()

100%|██████████| 100/100 [00:05<00:00, 18.26it/s]


In [11]:
df = comp.expected_risk_cost_df(t_end=200)
df.set_index(['failure_mode', 'task']).loc[['slow_aging', 'inspection']].head(10)

time cost cost_cumulative
failure_mode task                                
slow_aging   inspection    0    0               0
             inspection    1    0               0
             inspection    2    0               0
             inspection    3    0               0
             inspection    4    0               0
             inspection    5    0               0
             inspection    6    0               0
             inspection    7    0               0
             inspection    8    0               0
             inspection    9    0               0

In [8]:
dash_id = 'task-inspection-active'
dash_id = dash_id.replace(next_id + sep, "")
task_name= dash_id.split(sep)[0]
dash_id = dash_id.replace(next_id + sep, "")
dash_id

'inspection-active'

In [13]:
task.active

False

In [12]:
fm.tasks['inspection'].__dict__['active'] = False